In [1]:
import librosa
import keras
import os
import numpy as np
import scipy.io.wavfile as wav
import python_speech_features
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import math
from pydub import AudioSegment

Using TensorFlow backend.


In [26]:
path = "cut/28_uji.wav"
koef_mfcc = 13
audio_k, s_freq = librosa.load(path, sr=16000)
audio_duration =  librosa.get_duration(audio_k, sr=s_freq)
print(audio_duration)

4.9923125


In [7]:
def extract_data_test(start_duration, end_duration,i):
    path_new = "potongan/yafi"+str(i)+".wav"
    song = AudioSegment.from_file(path)
    song = song.set_frame_rate(16000)
    extract = song[start_duration*1000:end_duration*1000]
    extract.export(path_new, format="wav")
    #load wav
    sfreq, audio = wav.read(path_new)
    mfcc = python_speech_features.base.mfcc(audio,samplerate=16000,winlen=0.02,winstep=0.01,numcep=koef_mfcc,nfft=512,preemph=0.95,winfunc=np.hamming)
    #print(mfcc.shape)
    return mfcc

In [8]:
def create_model_1DCONV():
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=20, kernel_size=4, strides=1, padding="same", input_shape=[None, koef_mfcc]),
            keras.layers.Conv1D(filters=18, kernel_size=4, strides=1, padding="same"),
            keras.layers.Conv1D(filters=15, kernel_size=4, strides=1, padding="same"),
            keras.layers.Conv1D(filters=10, kernel_size=4, strides=1, padding="same"),
            keras.layers.GlobalMaxPooling1D(),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(3, activation='softmax')
            ])
    return model

In [24]:
model = create_model_1DCONV()
model.load_weights("conv1d_model_koef_"+str(koef_mfcc)+"_yafi.h5")

DURASI 1

In [27]:
kelas = ["nyala", "mati", "lain"]
kelas_threshold = 0.7
for i in range(1, math.ceil(audio_duration)+1):
    end_duration = i
    start_duration = i-1
    
    end_duration += 0.1
    start_duration-=0.01
    
    if start_duration<0 : start_duration = 0
    
    if end_duration>audio_duration : end_duration = audio_duration
    

    if end_duration>audio_duration : end_duration=audio_duration
    
    
    audio = extract_data_test(start_duration, end_duration, i)
    shape = audio.shape
    audio_p = audio.reshape(1, shape[0], shape[1])
    hasil = model.predict(audio_p)
    index_hasil = np.argmax(hasil)
    hasil = hasil.reshape(hasil.shape[-1])
    if hasil[index_hasil]<kelas_threshold:
        index_hasil = 2
                
    
    print("detik :"+str(start_duration)+"-"+str(end_duration)+" ="+ kelas[index_hasil])
        
        

detik :0-1.1 =nyala
detik :0.99-2.1 =lain
detik :1.99-3.1 =lain
detik :2.99-4.1 =lain
detik :3.99-4.9923125 =lain


Durasi 2

In [25]:
kelas = ["nyala", "mati", "lain"]
kelas_threshold = 0.7
for i in range(2, math.ceil(audio_duration)+1, 2):
    end_duration = i
    start_duration = i-2

    
    if start_duration<0 : start_duration = 0
    
    if end_duration>audio_duration : end_duration = audio_duration
    

    if end_duration>audio_duration : end_duration=audio_duration
    
    audio = extract_data_test(start_duration, end_duration, i)
    shape = audio.shape
    audio_p = audio.reshape(1, shape[0], shape[1])
    hasil = model.predict(audio_p)
    index_hasil = np.argmax(hasil)
    hasil = hasil.reshape(hasil.shape[-1])
    if hasil[index_hasil]<kelas_threshold:
        index_hasil = 2
                
    
    print("detik :"+str(start_duration)+"-"+str(end_duration)+" ="+ kelas[index_hasil])
        

detik :0-2 =nyala
detik :2-4 =lain
detik :4-6 =lain
detik :6-8 =lain
detik :8-9.984625 =lain
